# NOTE

The following code was created with reference to https://www.kaggle.com/code/vuppalaadithyasairam/baseline-cnn-for-radial-fracture-prediction
,special thanks

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from keras.preprocessing.image import ImageDataGenerator

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5

  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
generator = ImageDataGenerator(
    rescale = 1./255,
    validation_split = 0.2
)

In [3]:
Train_generator = generator.flow_from_directory(
    "/kaggle/input/bone-fracture-detection-using-xrays/archive (6)/train",
    target_size = (224, 224),
    batch_size = 16,
    class_mode ="binary",
    subset = "training" 
)

Validation_generator = generator.flow_from_directory(
    "/kaggle/input/bone-fracture-detection-using-xrays/archive (6)/train",
    target_size = (224, 224),
    batch_size = 16,
    class_mode ="binary",
    subset = "validation" 
)

Found 7091 images belonging to 2 classes.

Found 1772 images belonging to 2 classes.


In [4]:
Train_generator.next()[0].shape

(16, 224, 224, 3)

In [5]:
Validation_generator.next()[0].shape

(16, 224, 224, 3)

In [6]:
from tensorflow import keras

Retrain the Xception Model without any layer freezing. 

In [7]:
base_model = keras.applications.Xception(
    weights=None, 
    input_shape=(224, 224, 3),
    include_top=False)

In [8]:
base_model.trainable = True

In [9]:
inputs = keras.Input(shape=(224, 224, 3))

x = base_model(inputs, training=True)
x = keras.layers.GaussianNoise(0.25)(x)
x = keras.layers.GlobalAveragePooling2D()(x)
x = keras.layers.Dense(512,activation='relu')(x)
x = keras.layers.BatchNormalization()(x)
x = keras.layers.GaussianNoise(0.25)(x)
x = keras.layers.Dropout(0.25)(x)
outputs = keras.layers.Dense(1, activation = "sigmoid")(x)
model = keras.Model(inputs, outputs)

In [10]:
from keras import callbacks

In [11]:
earlyStopping = keras.callbacks.EarlyStopping(
    patience=7,
)

checkpoint_filepath = '/tmp/checkpoint'
model_checkpoint_callback = keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    monitor='val_accuracy',
    mode='max',
    save_best_only=True)

In [12]:
model.compile(optimizer="adam",
              loss="binary_crossentropy",
              metrics=["accuracy"])

In [13]:
model.fit(Train_generator, epochs=20, 
          callbacks=[earlyStopping,model_checkpoint_callback ],
          validation_data=Validation_generator)

Epoch 1/20

444/444 [==============================] - 135s 234ms/step - loss: 0.6446 - accuracy: 0.6552 - val_loss: 0.7023 - val_accuracy: 0.6620

Epoch 2/20

444/444 [==============================] - 71s 159ms/step - loss: 0.3695 - accuracy: 0.8443 - val_loss: 1.2254 - val_accuracy: 0.5418

Epoch 3/20

444/444 [==============================] - 71s 159ms/step - loss: 0.1909 - accuracy: 0.9277 - val_loss: 1.2328 - val_accuracy: 0.5756

Epoch 4/20

444/444 [==============================] - 71s 159ms/step - loss: 0.1193 - accuracy: 0.9595 - val_loss: 2.1279 - val_accuracy: 0.5102

Epoch 5/20

444/444 [==============================] - 71s 159ms/step - loss: 0.0909 - accuracy: 0.9698 - val_loss: 2.1228 - val_accuracy: 0.5023

Epoch 6/20

444/444 [==============================] - 74s 167ms/step - loss: 0.1548 - accuracy: 0.9427 - val_loss: 1.4969 - val_accuracy: 0.5801

Epoch 7/20

444/444 [==============================] - 73s 164ms/step - loss: 0.0776 - accuracy: 0.9728 - val_loss: 1

In [14]:
model.save("Test Model 2.h5")

In [15]:
Test_Generator = generator.flow_from_directory(
    "/kaggle/input/bone-fracture-detection-using-xrays/archive (6)/val",
    target_size = (224, 224),
    batch_size = 16,
    class_mode ="binary",
    subset = "training" 
)

Found 480 images belonging to 2 classes.


In [16]:
model.evaluate(Test_Generator)

30/30 [==============================] - 4s 123ms/step - loss: 1.6811 - accuracy: 0.6979


[1.681113600730896, 0.6979166865348816]